In [2]:
import os

from PIL import Image
from matplotlib.pyplot import imshow

import main
from Marble import Marble

MARBLE_BY_SYMBOL = dict(zip([Marble.symbol(e) for e in Marble], [e.name for e in Marble]))

status = main.init_image(Image.open(os.path.join("sample", "1.png")).convert('LA'))
print(status)


INFO:tensorflow:Restoring parameters from ./mnist_training/model.ckpt-1000


     e Q a v e a 
    - - - - a w - 
   - Q s f L s - - 
  - - w - a - w - - 
 - - e w s - e I a - 
f m Q - - G e - a - Q 
 f Q m w f - f f - v 
  S - w - w - m - a 
   e - f v a f T e 
    v - s - - w m 
     C - - - - e 


In [3]:
print("def neighbors")
x, y = 0, 0
result = []
for (dx, dy) in [(0, -1), (1, 0), (1, 1), (0, 1), (-1, 0), (-1, -1)]:
    n = (x + dx, y + dy)
    if n in status.state:
        result.append(status.state[n])
    else:
        result.append('-')
print(result)


def neighbors
['-', 'Q', '-', '-', '-', '-']


In [4]:
import numpy as np

from State import neighbors

print("def free")
neg = np.repeat(neighbors(status.state, x, y), 2)
xs = np.hstack(neg)
print(xs)
print(list(zip(*(xs[i:] for i in range(3)))))
print(('-', '-', '-') in list(zip(*(xs[i:] for i in range(3)))))


def free
['-' '-' 'Q' 'Q' '-' '-' '-' '-' '-' '-' '-' '-']
[('-', '-', 'Q'), ('-', 'Q', 'Q'), ('Q', 'Q', '-'), ('Q', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-')]
True


In [5]:
from State import free

print("def frees")
result = []
for (x, y) in status.state:
    if free(status.state, x, y):
        result.append((x, y))
print(result)


def frees
[(0, 0), (5, 0), (1, 2), (8, 4), (0, 5), (10, 5), (6, 9), (5, 10), (10, 10)]

In [6]:
from State import frees

def step(self, keyState):
    # for Quintessence use
    buckets = {}
    _frees = sorted([(Marble[MARBLE_BY_SYMBOL[self[x]]], x) for x in frees(self)])
    for (k, v) in _frees:
        buckets.setdefault(k, []).append(v)
    for a in _frees:
        (marbleA, posA) = a
        for b in _frees:
            (marbleB, posB) = b
            if a == b:
                continue
            elif marbleA.value in range(Marble.Salt.value, Marble.Earth.value + 1):
                if marbleB == marbleA or marbleB == Marble.Salt:
                    yield {posA, posB}
            elif marbleA.value in range(Marble.Vitae.value, Marble.Mors.value + 1):
                if marbleB.value in range(Marble.Vitae.value, Marble.Mors.value + 1) and marbleA != marbleB:
                    yield {posA, posB}
            elif marbleA.value in range(Marble.Tin.value, Marble.Silver.value + 1):
                if marbleB == Marble.Quicksilver and marbleA.previous() not in keyState:
                    yield {posA, posB}
            elif marbleA == Marble.Lead and marbleB == Marble.Quicksilver:
                yield {posA, posB}
            elif marbleA == Marble.Gold:
                yield {posA}
            elif marbleA == Marble.Quintessence:
                continue
            elif marbleA == Marble.Quicksilver:
                continue
            else:
                print(marbleA, marbleB)

list(step(status.state, status.keyState))


[{(5, 0), (6, 9)},
 {(5, 0), (8, 4)},
 {(6, 9), (8, 4)},
 {(5, 0), (8, 4)},
 {(0, 5), (6, 9)},
 {(0, 0), (6, 9)},
 {(0, 0), (10, 10)},
 {(6, 9), (10, 10)},
 {(0, 0), (10, 10)}]

In [27]:
from copy import deepcopy, copy

from State import State

def solve(status):
    todo = [status]
    print(todo)
    solutions = {str(status.state): []}
    while len(solutions) > 0:
        # eval
        cur_state = sorted(todo, key=lambda x: dict.__len__)[0]
        print(len(cur_state))
        todo.remove(cur_state)
        for _step in step(cur_state.state, status.keyState):
            state = deepcopy(cur_state.state)
            for pos in _step:
                if pos in state:
                    state.pop(pos)
                if str(state) in solutions:
                    continue
                todo += [State(state)]
                solution = copy(solutions[str(cur_state.state)])
                solution += _step
                solutions[str(state)] = solution
                if len(state) == 0:
                    return solution
    print(todo)
    return None


solve(status)


TypeError: object of type 'State' has no len()

In [26]:
from copy import deepcopy, copy

from State import State

def solve(status):
    todo = [status]
    solutions = {str(status.state): []}
    while len(solutions) > 0:
        # eval
        cur_state = sorted(todo, key=lambda x: len(x.state))[0]
        todo.remove(cur_state)
        for _step in step(cur_state.state, status.keyState):
            print(len(todo))
            state = deepcopy(cur_state.state)
            for pos in _step:
                if pos in state:
                    state.pop(pos)
                if str(state) in solutions:
                    continue
                todo += [State(state)]
                solution = copy(solutions[str(cur_state.state)])
                solution += _step
                solutions[str(state)] = solution
                if len(state) == 0:
                    print(solutions)
                    return solution
    return None


solve(status)


0
2
4
has key
5
has key
has key
5
has key
6
has key
7
9
has key
10
has key
has key
9
11
has key
has key
11
13
has key
14
has key
has key
14
16
has key
has key
16
has key
16
18
has key
has key
18


20
has key
21
has key
has key
21
23
has key
has key
23
has key
24
26
has key
has key
25
27
has key
28
has key
has key
28
30
has key
has key
30
has key
31
33
has key
has key
32
34
36
has key
has key
36
has key
has key
35
37
has key
38
has key
has key
38
40
has key
has key
40
has key
has key
39
41
43
has key
Marble.Lead Marble.Air
Marble.Lead Marble.Fire
Marble.Lead Marble.Water
Marble.Lead Marble.Earth
Marble.Lead Marble.Vitae
43
45
47
has key
Marble.Lead Marble.Copper
47
49
has key
has key
49
51
has key
has key
50
52
54
56
has key
57
has key
has key
57
has key
58
has key
59
has key
has key
59
has key
has key
59
61
has key
61
63
has key
has key
63
65
has key
has key
65
66
has key
66
has key
66
has key
66
has key
66
has key
66
has key
66
has key
66
has key
66
has key
66
has key
65
67
69
has key
70
has key
has key
70
72
73
has key
73
has key
73
has key
73
has key
73
has key
73
has key
73
has key
73
has key
73
has key
72
74
has key
has key
74
75
has key
75
has key
75
has key
75
has key
75


82
has key
82
has key
82
has key
81
83
has key
has key
83
has key
84
has key
has key
83
85
87
has key
has key
87
has key
88
has key
89
has key
has key
89
91
has key
has key
91
has key
has key
91
has key
has key
90
92
has key
93
has key
has key
93
has key
has key
93
has key
has key
93
has key
has key
89
has key
has key
89
has key
has key
89
has key
has key
89
has key
has key
89
has key
has key
89
has key
has key
88
90
has key
91
has key
has key
91
has key
has key
91
has key
has key
91
has key
has key
87
has key
has key
87
has key
has key
87
has key
has key
87
has key
has key
87
has key
has key
87
has key
has key
86
88
has key
has key
88
has key
has key
88
has key
has key
85
has key
has key
85
has key
has key
85
has key
has key
85
has key
has key
85
has key
has key
85
has key
has key
84
has key
85
has key
86
has key
has key
86
has key
has key
86
has key
87
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
82



78
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
78
has key
has key
78
has key
has key
78
has key
78
has key
78
has key
78
has key
78
has key
78
has key
78
has key
78
has key
78
has key
78
has key
77
has key
has key
77
has key
78
has key
has key
78
has key
79
has key
has key
79
has key
has key
79
80
has key
80
has key
80
has key
80
has key
80
has key
80
has key
80
has key
80
has key
80
has key
79
has key
has key
79
has key
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
78
80
has key
has key
80
81
has key
81
has key
81
has key
81
has key
81
has key
81
has key
81
has key
81
has key
81
has key
81
has key
80
82
has key
has key
82
84
has key
has key
84
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
85
has key
84
86
has key
has key
86
88
has key
has key
88
89
has key
89
has key
89
has key
89
has 

83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
82
84
86
has key
has key
86
has key
has key
86
has key
87
has key
has key
86
88
has key
has key
85
has key
has key
85
has key
has key
84
86
has key
has key
83
has key
has key
83
has key
has key
82
has key
has key
82
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
79
has key
has key
79
has key
has key
79
has key
has key
79
has key
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
79
has key
78
has key
79
has key
has key
79
81
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
79
has key
80
has key
has key
80
82
84
has key
has k


has key
has key
84
has key
has key
84
has key
has key
83
has key
has key
83
has key
has key
82
has key
has key
82
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
79
has key
80
has key
81
has key
has key
81
has key
has key
81
has key
82
has key
has key
82
has key
83
has key
84
has key
has key
84
has key
has key
84
has key
has key
84
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
83
has key
has key
82
has key
has key
82
has key
has key
82
has key
has key
82
has key
has key
82
has key
has key
82
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
81
has key
has key
80
has key
has key
80
has key
has key
80
has key
has key
80
has key
has k


99
has key
has key
99
has key
has key
99
has key
has key
98
has key
has key
98
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
96
98
has key
has key
98
has key
has key
98
has key
has key
98
has key
has key
98
has key
99
has key
has key
99
has key
has key
98
has key
has key
98
has key
has key
98
has key
has key
98
has key
99
has key
has key
99
has key
has key
99
101
has key
102
has key
has key
102
has key
has key
101
has key
has key
101
has key
has key
101
has key
has key
101
has key
has key
100
has key
101
has key
has key
101
has key
has key
101
has key
102
has key
has key
102
has key
has key
99
has key
has key
99
has key
has key
99
has key
has key
99
has key
has key
99
has key
has key
99
has key
has key
98
has key
has key
98
has key
has key
98
has key
has key
98
has key
99
has key
has key
99
has key
has key
97
has key
has key
97
has key
has key
97
has key
has key
97

KeyboardInterrupt: 